## Workflow to get features from a single game

Libraries needed and set up

In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
# root directory to the files
root_dir = "/content/drive/MyDrive/Diss/"

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sys
# path to tracknet model location
path = root_dir + "Tracknet/TrackNet_Three_Frames_Input/"
sys.path.append(path)
import Models

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
# path to mppe model location
path = root_dir + "MPPE/keras_Realtime_Multi-Person_Pose_Estimation/"
!pip install configobj
# Add the path to the library to sys.path
sys.path.append(path)
from processing import extract_parts, draw, extract_parts2

from config_reader import config_reader
from model.cmu_model import get_testing_model

## Get player location on frames where a ball exists

### Court class for converting 3d image to 2d top down view

In [ ]:
class Court:

    def __init__(self, front_wall, box, front_box):
        '''
        All coordinates must be a tuple
        where the list in index 0 are the X coordinates
        and index 1 are the Y coordinates

        all coordinate systems go top left, top right, bottom right, bottom left
        unless stated otherwise
        '''
        self.front_wallX, self.front_wallY = front_wall
        self.boxX, self.boxY = box
        self.front_boxX, self.front_boxY = front_box

        self.front_short_true = 5440
        self.box_true = 1600
        self.remainding_distance_true = 2610

        self.true_court_width = 762
        self.true_court_length = 975

        back_pointsL, back_pointsR = self.calc_back()

        self.final_cornersX = [self.front_wallX[0], self.front_wallX[1], back_pointsR[0], back_pointsL[0], self.front_wallX[0]]
        self.final_cornersY = [self.front_wallY[0], self.front_wallY[1], back_pointsR[1], back_pointsL[1], self.front_boxY[0]]

        # top left, top right, bottom left, bottom right
        self.final_corners = np.array([[self.final_cornersX[0], self.final_cornersY[0]],
                                       [self.final_cornersX[1], self.final_cornersY[1]],
                                        [self.final_cornersX[3], self.final_cornersY[3]],
                                         [self.final_cornersX[2], self.final_cornersY[2]]])
        # homography matrix
        self.M = self.create_hom_matrix()


    def create_hom_matrix(self):
      '''
      Creates a homogrophy matrix to translate the points to their real location
      '''


      # get x, y of feet position
      true_court_corners = np.array([[0, 0],
                                    [self.true_court_width, 0],
                                    [0, self.true_court_length],
                                    [self.true_court_width, self.true_court_length]])

      M, mask = cv2.findHomography(self.final_corners, true_court_corners)
      return M

    def use_matrix(self, x, y):
      '''
      Use the homography matrix on a set of coordinates
      '''

      denom = self.M[2,0] *x + self.M[2,1] * y + self.M[2,2]
      xPrime = (self.M[0,0] *x + self.M[0,1] * y + self.M[0,2]) / denom
      yPrime = (self.M[1,0] *x + self.M[1,1] * y + self.M[1,2]) / denom

      return int(xPrime), int(yPrime)

    def distance_ratio(self, dist, scale):
        return dist/scale

    def find_final_point(self, starting_point, further_point, distance):
        x1, y1 = starting_point
        x2, y2 = further_point

        # Calculate the distance between the two points
        distance_between_points = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        # Calculate the coordinates of the final point
        x = x2 + (distance * (x2 - x1)) / distance_between_points
        y = y2 + (distance * (y2 - y1)) / distance_between_points

        return x, y

    def calc_distance(self, X, Y):
        right = ((X[0] - X[3])**2 + (Y[0] - Y[3])**2)**0.5
        left = ((X[1] - X[2])**2 + (Y[1] - Y[2])**2)**0.5
        return left, right

    def calc_ratios(self, cornersX, cornersY, true_distance):

        distance_left, distance_right = self.calc_distance(cornersX, cornersY)

        ratio_left = self.distance_ratio(distance_left, true_distance)
        ratio_right = self.distance_ratio(distance_right, true_distance)

        return ratio_left, ratio_right

    def calc_back(self):

        # calculate the scale factors for different parts of the court
        front_short_ratioL, front_short_ratioR = self.calc_ratios(self.front_wallX, self.front_wallY, self.front_short_true)
        box_ratioL, box_ratioR = self.calc_ratios(self.boxX, self.boxY, self.box_true)

        # calculate scale factor for the final distance
        scale_factor_left = box_ratioL + (box_ratioL - front_short_ratioL)
        scale_factor_right = box_ratioR + (box_ratioR - front_short_ratioR)

        # calculate the final distance left
        remainding_distance_left = self.remainding_distance_true*scale_factor_left
        remainding_distance_right = self.remainding_distance_true*scale_factor_right

        # find the positions of the back points
        back_p_left = self.find_final_point((self.front_boxX[0], self.front_boxY[0]), (self.front_boxX[3], self.front_boxY[3]), remainding_distance_left)
        back_p_right = self.find_final_point((self.front_boxX[1], self.front_boxY[1]), (self.front_boxX[2], self.front_boxY[2]), remainding_distance_right)

        return back_p_left, back_p_right

An example of a court

In [ ]:
# cornersX = [630, 1320, 1485, 457, 630]
# cornersY = [517, 523, 738, 730, 517]

# serv_boxX = [457, 1485, 1573, 367, 457]
# serv_boxY = [730, 738, 853, 840, 730]

# serv_frontX = [630, 1320, 1573, 367, 630]
# serv_frontY = [517, 523, 853, 840, 517]

# court = Court((cornersX, cornersY), (serv_boxX, serv_boxY), (serv_frontX, serv_frontY))

### Set up multi-person pose estimates

In [ ]:
class MPPE:

  def __init__(self, path):

    # model
    self.m = get_testing_model()
    # load weights
    self.m.load_weights(f'{path}/model/keras/model.h5')
    self.params, self.model_params = config_reader(f'{path}/config')



In [ ]:
# input is mppe model location path
mppe_model = MPPE(root_dir + "MPPE/keras_Realtime_Multi-Person_Pose_Estimation")

In [ ]:
def mppe(image, params, model, model_params):
    input_image = image

    body_parts, all_peaks, subset, candidate = extract_parts(input_image, params, mppe_model.m, model_params)
    canvas = draw(input_image, all_peaks, subset, candidate)

    return canvas

In [ ]:
def visualise(cropped_frame, bl, br):
  # plot people on graph
  body_parts, all_peaks, subset, candidate = extract_parts(cropped_frame, mppe_model.params, mppe_model.m, mppe_model.model_params)

  positions = []
  body_parts_list = []
  for i, key in zip(range(18), body_parts.keys()):
      # list to hold limbs already plotted
      plotted_list = []
      if len(all_peaks[i])>1:
          sorted_peaks = sorted(all_peaks[i], key=lambda x: x[2])
      elif len(all_peaks[i]) == 0:
          continue
      else:
          sorted_peaks = all_peaks[i]
      a = sorted_peaks[0][0]
      b = sorted_peaks[0][1]
      adj_a, adj_b = bl+a, br+b
      if (adj_a, adj_b) not in plotted_list:
          plotted_list.append((adj_a,adj_b))
      else:
          a = sorted_peaks[1][0]
          b = sorted_peaks[1][1]
          adj_a, adj_b = bl+a, br+b

      if body_parts[key] != (None, None):
            body_parts_list.append(key)
      positions.append([adj_a,adj_b])

  return np.array(positions), body_parts_list

In [ ]:
def pose_estimates(frame, boxes, body_part_dict, court):
  feetpositions = [[None,None],[None, None]]
  # for each player found on the image
  for j, box in enumerate(boxes):
    bl, br, tl, tr = tensor_toint(box)
    cropped_frame = frame[br:tr, bl:tl]

    h,w,_ = cropped_frame.shape
    # get the positions for body parts for each person detected
    positions, body_parts = visualise(cropped_frame, bl, tr-h)

    for i, label in enumerate(body_parts):
      body_part_dict[label] = positions[i, 0], positions[i, 1]

    right_foot_coords = body_part_dict['right_ankle']
    left_foot_coords = body_part_dict['left_ankle']

    # get feet positions by finding midpoint between ankles
    feet_pos = midpoint_single(right_foot_coords, left_foot_coords)

    # if the feet are found add their positions
    if feet_pos:
      # translate the points to their real location
      feet_pos = court.use_matrix(*feet_pos)
      feetpositions[j] = [feet_pos[0], feet_pos[1]]

  return feetpositions

### Find feet positions

In [ ]:
def midpoint_single(point1, point2):
    if point1 and point2:
        x1, y1 = point1
        x2, y2 = point2
        # Both variables are present, calculate midpoint
        midpoint = ((x1 + x2) / 2, (y1 + y2) / 2)
        return midpoint
    elif point1:
        # Only var1 is present
        return point1
    elif point2:
        # Only var2 is present
        return point2
    else:
        # Both variables are None
        return None

Test feet positions are relocated correctly

In [ ]:
def test_feet_pos():
  # set up player tracker
  body_parts_list = ['nose', 'neck', 'right_shoulder', ' right_elbow', 'right_wrist', 'left_shoulder', 'left_elbow', 'left_wrist', 'right_hip', 'right_knee', 'right_ankle', 'left_hip', 'left_knee', 'left_ankle', 'right_eye', 'left_eye', 'right_ear', 'left_ear']
  body_part_dict = {}
  for part in body_parts_list:
    body_part_dict[part] = None


  cap = cv2.VideoCapture(root_dir + "Videos/test_edit.mp4")

  ret, img = cap.read()



  if ret:
    plt.imshow(img)
    boxes = player_finder(img)
    feets = pose_estimates(img, boxes, body_part_dict)

  return feets

In [ ]:
#test_feet_pos()

In [ ]:
# squash_court = ([0, 762, 762, 0, 0], [0, 0, 975, 975, 0])
# short_line = ([0, 762], [544, 544])
# half_court_line = ([762//2, 762//2], [544, 975])
# box1 = ([0, 160, 160, 0], [544, 544, 544+160, 544+160])
# box2 = ([762, 762-160, 762-160, 762], [544, 544, 544+160, 544+160])

# plt.figure(figsize=(7.62/2, 9.75/2))
# plt.plot(*squash_court, c='red')
# plt.plot(*short_line, c='red')
# plt.plot(*half_court_line, c='red')
# plt.plot(*box1, c='red')
# plt.plot(*box2, c='red')
# plt.scatter(*use_mask(*feetpositions[0][0]))
# plt.scatter(*use_mask(*feetpositions[0][1]))
# plt.show()

### YOLO model

In [ ]:
yolo_model = YOLO(root_dir + "yolov8n.pt")

In [ ]:
def tensor_toint(tensors):
  '''
  Converts a list of tensor coords to integers
  '''
  tensor_ints = []
  for tensor in tensors:
      tensor_ints.append(int(tensor))
  return tensor_ints

In [ ]:
def player_finder(frame):
  # Run YOLOv8 tracking on the frame
  results = yolo_model(frame, conf=0.2, max_det=2, classes=0)

  # Get the boxes
  boxes = results[0].boxes.xyxy.cpu()

  return boxes



## Tracknet Model

In [ ]:
def load_tracknet():
  height = 360
  width = 640
  n_classes = 256
  modelFN = Models.TrackNet.TrackNet
  m = modelFN( n_classes , input_height=height, input_width=width   )
  m.compile(loss='categorical_crossentropy', optimizer= 'adadelta' , metrics=['accuracy'])
  # input is path to tracknet model weights
  m.load_weights(root_dir + "Tracknet/TrackNet_Three_Frames_Input/weights/squash_model.h5")
  return m

In [ ]:
tracknet_model = load_tracknet()

layer24 output shape: 256 360 640


## Get feet positions and ball position

In [ ]:
def get_ball_pos(img, img1, img2, ball_tracker):

  output_img = img

  #resize it
  img = cv2.resize(img, ( ball_tracker.width , ball_tracker.height ))
  #input must be float type
  img = img.astype(np.float32)


  #combine three imgs to  (width , height, rgb*3)
  X =  np.concatenate((img, img1, img2),axis=2)

  #since the odering of TrackNet  is 'channels_first', so we need to change the axis
  X = np.rollaxis(X, 2, 0)
  #prdict heatmap
  pr = tracknet_model.predict( np.array([X]) )[0]

  #since TrackNet output is ( net_output_height*model_output_width , n_classes )
  #so we need to reshape image as ( net_output_height, model_output_width , n_classes(depth) )
  #.argmax( axis=2 ) => select the largest probability as class
  pr = pr.reshape(( ball_tracker.height ,  ball_tracker.width , ball_tracker.n_classes ) ).argmax( axis=2 )

  #cv2 image must be numpy.uint8, convert numpy.int64 to numpy.uint8
  pr = pr.astype(np.uint8)

  #reshape the image size as original input image
  heatmap = cv2.resize(pr  , (ball_tracker.output_width, ball_tracker.output_height ))

  #heatmap is converted into a binary image by threshold method.
  ret,heatmap = cv2.threshold(heatmap,127,255,cv2.THRESH_BINARY)

  #find the circle in image with 2<=radius<=7
  circles = cv2.HoughCircles(heatmap, cv2.HOUGH_GRADIENT,dp=1,minDist=1,param1=50,param2=2,minRadius=2,maxRadius=7)

  #check if there have any tennis be detected
  if circles is not None:
    #if only one tennis be detected
    if len(circles) == 1:

      x = int(circles[0][0][0])
      y = int(circles[0][0][1])

      return x, y

  return None


In [ ]:
class Ball_Tracker2:

  def __init__(self):

    self.width, self.height = 640, 360
    self.n_classes = 256

    self.output_width = None
    self.output_height = None


  def process_first_two(self, video):
    '''
    process first two frames as ball tracker won't work for the first two
    '''

    currentFrame = 0

    self.output_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    self.output_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    img, img1 = None, None

    video.set(1,currentFrame);
    ret, img1 = video.read()

    currentFrame +=1
    #resize it
    img1 = cv2.resize(img1, (self.width , self.height))
    #input must be float type
    img1 = img1.astype(np.float32)

    #capture frame-by-frame
    video.set(1,currentFrame);
    ret, img = video.read()

    currentFrame +=1
    #resize it
    img = cv2.resize(img, ( self.width , self.height ))
    #input must be float type
    img = img.astype(np.float32)

    return img, img1



In [ ]:
def main(name, court):

  # load video
  video_path = root_dir + "Data/" + name + ".mp4"
  cap = cv2.VideoCapture(video_path)

  # set up ball tracker
  ball_tracker = Ball_Tracker2()
  img1, img2 = ball_tracker.process_first_two(cap)

  # set up player tracker
  body_parts_list = ['nose', 'neck', 'right_shoulder', ' right_elbow', 'right_wrist', 'left_shoulder', 'left_elbow', 'left_wrist', 'right_hip', 'right_knee', 'right_ankle', 'left_hip', 'left_knee', 'left_ankle', 'right_eye', 'left_eye', 'right_ear', 'left_ear']
  body_part_dict = {}
  for part in body_parts_list:
    body_part_dict[part] = None

  feetpositions = []

  ball_positions = []

  # set current frame
  current_frame = 1

  while True:
    current_frame += 1

    cap.set(1,current_frame)

    ret, img = cap.read()

    if not ret:
      break

    ball_pos = get_ball_pos(img, img1, img2, ball_tracker)
    # if a ball is detected, get ball coordinates and do player position
    if ball_pos:
      ball_pos = court.use_matrix(*ball_pos)
      ball_x, ball_y = ball_pos

      # extract player locations
      boxes = player_finder(img)

      # find player pose estimates and add their feet positions
      feets = pose_estimates(img, boxes, body_part_dict, court)
      # check if there are two feet detected
      if [None, None] not in feets:
        feetpositions.append(feets)

        ball_positions.append([ball_x, ball_y])

        print(feetpositions[-1], ball_positions[-1])

        squash_court = ([0, 762, 762, 0, 0], [0, 0, 975, 975, 0])
        short_line = ([0, 762], [544, 544])
        half_court_line = ([762//2, 762//2], [544, 975])
        box1 = ([0, 160, 160, 0], [544, 544, 544+160, 544+160])
        box2 = ([762, 762-160, 762-160, 762], [544, 544, 544+160, 544+160])

        # plt.figure(figsize=(7.62/2, 9.75/2))
        # plt.plot(*squash_court, c='red')
        # plt.plot(*short_line, c='red')
        # plt.plot(*half_court_line, c='red')
        # plt.plot(*box1, c='red')
        # plt.plot(*box2, c='red')
        # plt.scatter(*feetpositions[-1][0])
        # plt.scatter(*feetpositions[-1][1])
        # plt.show()

  return feetpositions, ball_positions


In [ ]:
def get_euclidean(p1, p2):
  p1, p2 = np.array(p1), np.array(p2)
  square = np.square(p1 - p2)
  sum_square = np.sum(square)
  return np.sqrt(sum_square)

In [ ]:
def distance_wall(feetposition):
  '''
  Outputs the distances as fl, fr, bl, br
  '''
  # true court coordinates
  # front_left, front_right, back_left, back_right
  court_coords = [(0, 0), (762, 0), (0, 975), (762, 975)]
  distances = []
  for coord in court_coords:
    distances.append(get_euclidean(feetposition, coord))
  return distances

In [ ]:
def test_distances_wall():
  feet_pos = (0,0)
  distances = distance_wall(feet_pos)
  assert distances == [0.0, 762.0, 975.0, 1237.444544211982]
test_distances_wall()

In [ ]:
def create_features(feetposition, ballposition):
  '''
  returns an numpy array of all the features
  '''
  # get distance between players
  player_distance = [get_euclidean(feetposition[0], feetposition[1])]

  # player court distances and distances from each player to each corner
  player_court = np.array([distance_wall(player) for player in feetposition]).flatten().tolist()

  # ball to wall
  ball_wall = distance_wall(ballposition)

  # get distance between players and ball
  player_ball = [None, None]
  for i, player in enumerate(feetposition):
    player_ball[i] = get_euclidean(player, ballposition)

  final_list = [player_distance, player_court, player_ball, ball_wall]
  combined_list = [item for sublist in final_list for item in sublist]
  print(combined_list)

  # flatten list and return as np array
  return combined_list

In [ ]:
def create_all_features(feetpositions, ballpositions, fith=True):
  if fith:
    # take every 5th frame starting from the back for 5 frames

    frames_skip = 5
    frame_cap = 5

    # reverse list then truncate
    fPos_use = feetpositions[::-frames_skip][:frame_cap]
    bPos_use = ballpositions[::-frames_skip][:frame_cap]

    # final position would be first in new list
    final_ball_pos = bPos_use[0]
    final_player_pos = fPos_use[0]

  features = []

  for fPos, bPos in zip(feetpositions, ballpositions):
    feature = create_features(fPos, bPos)
    features.append(feature)

  return features

In [ ]:
import os
import pandas as pd

In [ ]:
courts = pd.read_csv(root_dir + 'court_pos.csv')

In [ ]:
def get_x(x):
    x = x.split(',')
    return int(x[0][1:])

def get_y(x):
    x = x.split(',')
    return int(x[1][:-1])

def get_x_y(cols, name):
  '''
  Splits x and y out for the column
  '''
  X = []
  Y = []
  row = courts.loc[courts['name'] == name].index[0]
  startX, startY = None, None
  for col in cols:
    x = get_x(courts[col][row])
    y = get_y(courts[col][row])
    X.append(x)
    Y.append(y)
    if not startX:
      startX, startY = x, y

  X.append(startX)
  Y.append(startY)
  return X, Y

## Create the features for the dataset, make sure the folder names are correct

Also make sure there is lets/, no-lets/ and strokes/ in Data/

In [ ]:
def create_all():
  # key point names for court transformation
  positions = [['top left', 'top right', 'serv right', 'serv left'],
               ['serv left', 'serv right', 'box right', 'box left'],
               ['top left', 'top right', 'box right', 'box left']]
  folders = ['lets', 'no-lets', 'strokes']
  all_features = []
  for folder in folders:
    files = os.listdir(root_dir + 'Data/' + folder)
    for f in files:
      name = f.split('.')[0]
      # get positions of key points
      pos = [get_x_y(position, name) for position in positions]
      court = Court(*pos)
      f, b = main(folder + '/' + name, court)
      all_features.append(create_all_features(f, b, False))


  return all_features



In [ ]:
all_features = create_all()

In [ ]:
headers = ['Player Distance', 'P1 f-left', 'P1 f-right', 'P1 b-left', 'P1 b-right',
           'P2 f-left', 'P2 f-right', 'P2 b-left', 'P2 b-right', 'P1 ball', 'P2 ball',
           'B f-left', 'B f-right', 'B b-left', 'B b-right', 'video id', 'result']

In [ ]:
final_df = pd.DataFrame(columns=headers)

In [ ]:
video_id = 0
folders = ['lets', 'no-lets', 'strokes']
for folder in folders:
    files = os.listdir(root_dir + 'Data/' + folder)
    for f in files:
      decision = folders.index(folder)
      for feature in all_features[video_id]:
        final_feature = feature + [video_id, decision]
        #print(final_feature)
        new_df = pd.DataFrame([final_feature], columns=headers)
        final_df = pd.concat([final_df, new_df], ignore_index=True)
      video_id += 1

In [ ]:
final_df.head()

,Player Distance,P1 f-left,P1 f-right,P1 b-left,P1 b-right,P2 f-left,P2 f-right,P2 b-left,P2 b-right,P1 ball,P2 ball,B f-left,B f-right,B b-left,B b-right,video id,result
0,168.002976,715.629094,768.016276,473.814310,549.749034,654.369926,872.900911,377.524834,690.131147,1715.270241,1685.634895,1039.931248,1314.974144,2014.480578,2169.292972,0,0
1,170.575496,724.572288,772.420222,470.935240,541.671487,650.028461,867.896883,382.899465,690.883492,1676.702717,1633.045009,991.889611,1276.125778,1966.448575,2124.046139,0,0
2,156.003205,715.712931,764.115829,478.037655,547.857646,656.588151,860.511476,386.436282,677.277639,1575.310128,1545.032362,898.761926,1202.000416,1873.365421,2036.303023,0,0
3,146.054784,713.011220,763.584311,478.079491,550.668684,654.006881,851.413531,394.588393,672.963595,1545.007443,1513.492649,871.516494,1177.244664,1846.243754,2008.725964,0,0
4,137.295302,717.864890,769.109875,473.080331,547.727122,657.282283,846.763249,397.548739,665.607242,1432.456980,1395.609186,750.485843,1088.586698,1725.211291,1896.877961,0,0


In [ ]:
final_df.to_csv("final_dataset.csv", index=False)